In [ ]:
#import the dataset
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset
!unzip new-plant-diseases-dataset.zip

In [ ]:
#install the required dependencies
!pip install evaluate
!pip install accelerate -U
!pip install datasets evaluate matplotlib
!pip install torch torchvision transformers

In [ ]:
#import the necessary modules
import PIL
import torch
import random
import evaluate
import warnings
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import AutoImageProcessor
from datasets import load_metric,list_metrics
from datasets import load_dataset,concatenate_datasets
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomVerticalFlip,
    GaussianBlur,
    RandomResizedCrop,
    Resize,
    ToTensor,
)
warnings.filterwarnings('ignore')

In [ ]:
#load the dataset
train_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
val_dir = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

dataset_t = load_dataset("imagefolder", data_dir=train_dir,drop_labels=False)
dataset_v = load_dataset("imagefolder", data_dir=val_dir,drop_labels=False)

In [ ]:
metrics_list = list_metrics()
metric = load_metric("accuracy") #load the accuracy metric from the datset module

In [ ]:
#mapping the labels to ID(integer)
labels = dataset_t["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

labels = dataset_v["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

#print(len(dataset_t["train"]['label']))
#print(len(dataset_v["train"]['label']))

In [ ]:
model_checkpoint = "google/vit-base-patch16-224-in21k" #using ViT, a pre-trained model, trained on ImageNet-21k dataset
image_processor  = AutoImageProcessor.from_pretrained(model_checkpoint) # config for preprocessing images

In [ ]:
normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose(
        [
            RandomResizedCrop(crop_size),
            RandomHorizontalFlip(),
            RandomVerticalFlip(),
            GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.)),
            ToTensor(),
            normalize,
        ]
    )
train_transforms_without_tensor = Compose(
        [
            RandomResizedCrop(crop_size),
            RandomHorizontalFlip(),
            RandomVerticalFlip(),
            GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.)),
        ]
    )
train_transforms_keep_original = Compose(
        [
            Resize(crop_size),
            ToTensor(),
            normalize,
        ]
    )
train_transforms_keep_original_without_tensor = Compose(
        [
            Resize(crop_size),
        ]
    )
val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(crop_size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    example_batch["image_transform"] = [
        train_transforms_without_tensor(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_original(example_batch):
    example_batch["pixel_values"] = [
        train_transforms_keep_original(image.convert("RGB")) for image in example_batch["image"]
    ]
    example_batch["image_transform"] = [
        train_transforms_keep_original_without_tensor(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [ ]:
train_ds = dataset_t["train"]
val_ds = dataset_v["train"]

In [ ]:
def showDistribution():
    distribution = evaluate.load("label_distribution")
    fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(15,5))

    ax[0].set_title("Training dataset " + str(len(train_ds)))
    results = distribution.compute(data=train_ds['label'])
    ax[0].bar(results['label_distribution']['labels'], results['label_distribution']['fractions'])

    ax[1].set_title("Validation dataset " + str(len(val_ds)))
    results = distribution.compute(data=val_ds['label'])
    ax[1].bar(results['label_distribution']['labels'], results['label_distribution']['fractions'])
    plt.show()
showDistribution()

In [ ]:
# we will set the transformation method to the list.
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [ ]:
num = int(random.random() * len(dataset_t["train"]['label']))
#print(num)
def showimages(dataset,numberofimg):
    sorted_ds = dataset.sort('label')
    samples = sorted_ds.select(range(num, num+numberofimg))
    pointer = 0
    fig, ax = plt.subplots(5, 3, sharex=True, sharey=True, figsize=(10,6))
    for i in range(5):
        for j in range(3):
            ax[i,j].imshow(samples[pointer]['image_transform'])
            ax[i,j].set_title(f"{labels[samples[pointer]['label']]}")
            ax[i,j].axis('off')
            pointer+=1
    plt.show()
showimages(train_ds,15)

In [ ]:
model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,
    id2label=id2label, # aligning model's output to class labels in the dataset
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

In [ ]:
# Use GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#print(device)
model.to(device)


In [ ]:
batch_size = 32
args = TrainingArguments(
    remove_unused_columns=False,
    evaluation_strategy="epoch", #evaluate at each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4, #gradients over 4 steps before updating the model's weights
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    warmup_ratio=0.1, #fraction of total steps to use for learning rate warmup
    logging_steps=10, #Logs training progress every 10 steps
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    output_dir='./results', #Directory for saving files
    resume_from_checkpoint=True,
    overwrite_output_dir=True,
    run_name="Disease_classification", # to avoid warning
)

In [ ]:
# Computes accuracy on a batch of predictions made by a model
def compute_metrics(eval_pred): # takes a Named Tuple as input
    predictions = np.argmax(eval_pred.predictions, axis=1) # predictions are the logits of the model as Numpy arrays,
    return metric.compute(predictions=predictions, references=eval_pred.label_ids) # label_ids are the ground-truth labels as Numpy arrays.

In [ ]:
# organizing data into a format suitable for training
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
# Setting the trainer
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn
)

In [ ]:
train_results = trainer.train() # Start the training

In [ ]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics) # Logs the metrics
trainer.save_metrics("eval", metrics) # Saves the metrics

In [ ]:
model.eval() #to set dropout and batch normalization layers to evaluation mode

test_dir = '/content/test/test/'
dataset_t = load_dataset("imagefolder", data_dir=test_dir,drop_labels=False)

labels = dataset_test["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label
test_ds = dataset_test["train"]

data = iter(test_ds)
images, labels = next(data)
outputs = model(images)

correct = 0
total = 0
# for testing, no need to calculate the gradients for outputs
with torch.no_grad():
    for data in test_ds:
        images, labels = data
        outputs = model(images)
        # the class with the highest confidence is the prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy : {100 * correct // total} %')
